# BEEF example: features

In [1]:
from beef import fe
import beef
from beef.newmark import factors as newmark_factors, factors_from_alpha

import numpy as np

from vispy import app, gloo
app.use_app('ipynb_webgl')
import matplotlib.pyplot as plt

In [2]:
# Define sections
beam = dict(
    A=0.1,
    m=5000,
    I_z=1e-2,
    I_y=1e-2,
    E=210e9,
    J=1e-2,
    poisson=0.3
    )

beam_section = fe.Section(**beam, name='Beam section')

In [3]:
# Define geometry and mesh
x1 = np.linspace(0,100,10)
x2 = np.linspace(110,210,10)
y1 = x1*0
y2 = x1*0
z = x1*0

node_labels1 = np.arange(101,110+1)
node_labels2 =  np.arange(201,210+1)

node_matrix1 = np.vstack([node_labels1, x1, y1, z]).T
node_matrix2 = np.vstack([node_labels2, x2, y2, z]).T

element_matrix1 = np.vstack([node_labels1[:-1], node_labels1[:-1], node_labels1[1:]]).T
element_matrix2 = np.vstack([node_labels2[:-1], node_labels2[:-1], node_labels2[1:]]).T

sections1 = [beam_section]*element_matrix1.shape[0]
sections2 = [beam_section]*element_matrix2.shape[0]

In [4]:
# Define constraints, features and assembly
constraints = [fe.Constraint([101], dofs='all', node_type='beam3d', name='Fixed end'),  # fixed at left end of left part
               fe.Constraint([110,201,210], dofs=[1,2,3], node_type='beam3d', name='Fixed end')] # simply supported and torsionally supported at other three ends

spring_nodelabels = [110, 201]
spring_dofs = [0]
spring_val = 1e2

# mass_nodelabel = [206]
# mass_dofs = [0,1,2]
# mass_val = 0e3  # 5 tonnes

features = [fe.Spring(spring_nodelabels, spring_dofs, spring_val)]

part1 = fe.Part(node_matrix1, element_matrix1, sections1)
part2 = fe.Part(node_matrix2, element_matrix2, sections2)

assembly = fe.Assembly([part1, part2], constraints=constraints, features=features)
assembly.plot(node_labels=True, element_labels=False, plot_nodes=True)
k_features = assembly.get_feature_mats(['k'])


VispyWidget(height=600, width=800)

In [14]:
# Create analysis: linear static
force_amplitude = 12e3 # 12 kN
forces = [fe.Force([205], 0, force_amplitude)]
analysis = fe.Analysis(assembly, forces=forces)
analysis.run_lin_static()

analysis.eldef.plot(overlay_deformed=True, plot_nodes=False, node_labels=False)
print(f'k_target = {spring_val:.3e}, k_FE = {force_amplitude/analysis.u[-6,-1]:.3e}')

Static analysis:   0%|          | 0/2 [00:00<?, ?it/s]

VispyWidget(height=600, width=800)

k_target = 1.000e+02, k_FE = 1.000e+02


In [6]:
# Create analysis: eigenvalue solution
from beef.modal import normalize_phi

analysis = fe.Analysis(assembly)
lambd, phi = analysis.run_eig()

phi = normalize_phi(phi)

mode_ix = 1
scaling = 20.0
analysis.eldef.deform(phi[:,mode_ix]*scaling)
print(f'Mode {mode_ix+1}, f = {np.abs(np.imag(lambd[mode_ix])/2/np.pi):.4f} Hz')
analysis.eldef.plot(overlay_deformed=True)

Mode 2, f = 0.1018 Hz


VispyWidget(height=600, width=800)

(<SceneCanvas (ipynb_webgl) at 0x2264ecdabe0>, <ViewBox at 0x22653e42580>)